<a href="https://colab.research.google.com/github/yogivirat/MATH-509-FINAL-PROJECT/blob/master/Math_509_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MATH 509 Final Project

### Project Description

In today's fast-paced digital landscape, finding personalized recommendations for
books, movies, games, and social media influencers can be overwhelming. Our project aims to
address this challenge by developing a user-friendly Recommender System (RS) that simplifies
the process of discovering enjoyable content across various platforms. By leveraging
Collaborative Filtering (CF) and Matrix Factorization (MF) techniques, our RS will provide
fast and accurate recommendations tailored to individual preferences.
List of Goals:
1. Develop an efficient and scalable RS capable of handling large datasets and delivering
real-time recommendations to users.
2. Incorporate Collaborative Filtering algorithms, including Memory-Based Collaborative
Filtering and Model-Based Collaborative Filtering, to analyze user preferences and
generate personalized recommendations.
3. Utilize Matrix Factorization techniques to learn latent user preferences and item
attributes, thereby enhancing the accuracy of recommendations.
4. Design an intuitive user interface that enables users to interact effortlessly with the RS
and receive personalized recommendations across different content categories.
5. Fine-tune the performance of the RS to ensure swift response times and efficient
resource utilization, thereby providing users with a seamless experience.
6. Conduct thorough evaluations and testing to gauge the effectiveness and accuracy of
the RS in generating relevant recommendations for users.
By achieving these goals, our project aims to simplify the process of discovering enjoyable
content for users while making a valuable contribution to the field of Recommender Systems

Data Set Link: https://grouplens.org/datasets/hetrec-2011/ || https://drive.google.com/drive/folders/1YPYlOG7xbTqQ8jy5IetnL7beO64hKhvG?usp=sharing


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import string
import re
import matplotlib.pyplot as plt
import math
from matplotlib import rc
from google.colab import drive
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
import matplotlib.ticker as ticker
from math import sqrt


from sklearn.metrics import mean_squared_error

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [ ]:
from sklearn.metrics import mean_squared_error

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.3)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42


In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
plays = pd.read_csv('/content/drive/My Drive/final_dataSet/user_artists.dat', sep='\t')


In [10]:
artists = pd.read_csv('/content/drive/My Drive/final_dataSet/artists.dat', sep='\t', usecols=['id','name'])

In [11]:
ap = pd.merge(
  artists, plays,
  how="inner",
  left_on="id",
  right_on="artistID"
)

In [12]:
ap = ap.rename(columns={"weight": "playCount"})

In [13]:
ap.head()

,id,name,userID,artistID,playCount
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152


## EXPLORATION

In [ ]:
artist_rank = ap.groupby(['name']) \
  .agg({'userID' : 'count', 'playCount' : 'sum'}) \
  .rename(columns={"userID" : 'totalUniqueUsers', "playCount" : "totalArtistPlays"}) \
  .sort_values(['totalArtistPlays'], ascending=False)

artist_rank['avgUserPlays'] = artist_rank['totalArtistPlays'] / artist_rank['totalUniqueUsers']

In [ ]:
ap = ap.join(artist_rank, on="name", how="inner") \
  .sort_values(['playCount'], ascending=False)

In [ ]:
ap.head()

In [ ]:
def bar_chart_int(x, y, x_label, y_label, title, caption, total_val):
    fig, ax = plt.subplots()
    fig.set_size_inches(16, 5)
    ax = sns.barplot(x=x[:20], y=y[:20], palette='Blues_r')  # Adjusted to use keyword arguments
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, p: '{:,}'.format(int(x))))

    # our bar label placement
    for p in ax.patches:
        height = p.get_height()
        pct = 100*(height/total_val)
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.1f}%'.format(pct),
                ha="center",verticalalignment='bottom',color='black', fontsize=12) 

    # our caption statement
    ax.text(19, max(y[:20])*0.95, caption,horizontalalignment='right')

    plt.xticks(rotation=90)
    plt.show()

In [ ]:
artist_rank.head()

In [ ]:
ap.head()

In [ ]:
ax = ap.playCount.value_counts().hist(bins=100)
ax.set_xlim((0, 200))
ax.set_title("Artist played count")
ax.set_xlabel("user count")
ax.set_ylabel("played times");

In [ ]:
c2 = artist_rank.sort_values(['totalUniqueUsers'],ascending=False)
x = c2.index
y = c2.totalUniqueUsers
x_label = 'Artist Name'
y_label = 'Unique Users Played'
title = 'Unique Users per Artist'
caption = 'Percentage of total unique users'
total_val = ap.userID.nunique()

bar_chart_int(x,y,x_label,y_label,title,caption,total_val)

In [ ]:
c1 = artist_rank
x = c1.index
y = c1.totalArtistPlays
x_label = 'Artist Name'
y_label = 'Total Artist Plays'
title = 'Total Plays by Artist'
caption = 'Percentage of total plays'
total_val = c1.totalArtistPlays.sum()

bar_chart_int(x,y,x_label,y_label,title,caption,total_val)

In [ ]:
top_artists = artist_rank.sort_values(['totalArtistPlays'], ascending=False).index[:12]

x = artist_rank.totalUniqueUsers
y = artist_rank.totalArtistPlays
labels = artist_rank.index

fig, ax = plt.subplots(figsize=(15,10))
sns.regplot(x=x, y=y, ax=ax)  # Pass x and y as keyword arguments
ax.set_title('Artist Popularity: Play Count vs Unique Users')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:,.0f}'.format(y)))
ax.set_xlabel('Total Unique Users')
ax.set_ylabel('Total Artist Plays')

for i, t in enumerate(labels):
    if t in top_artists:
        ax.annotate(t, (x[i], y[i]))

plt.show()

In [ ]:
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())

ap = ap.assign(playCountScaled=play_count_scaled)

In [ ]:
ratings_df = ap.pivot(
    index='userID', 
    columns='artistID', 
    values='playCountScaled'
)

In [ ]:
ratings = ratings_df.fillna(0).values

In [ ]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('{:.2f}%'.format(sparsity))

In [ ]:
MIN_USER_RATINGS = 35
DELETE_RATING_COUNT = 15

def train_test_split(ratings):
    
    validation = np.zeros(ratings.shape)
    train = ratings.copy()
    
    for user in np.arange(ratings.shape[0]):
        if len(ratings[user,:].nonzero()[0]) >= MIN_USER_RATINGS:
            val_ratings = np.random.choice(
                ratings[user, :].nonzero()[0], 
                size=DELETE_RATING_COUNT,
                replace=False
            )
            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]
    return train, validation

In [ ]:
train, val = train_test_split(ratings)


In [ ]:
train.shape

In [ ]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [ ]:
class Recommender:
  
  def __init__(self, n_epochs=200, n_latent_features=3, lmbda=0.1, learning_rate=0.001):
    self.n_epochs = n_epochs
    self.n_latent_features = n_latent_features
    self.lmbda = lmbda
    self.learning_rate = learning_rate
  
  def predictions(self, P, Q):
    return np.dot(P.T, Q)
  
  def fit(self, X_train, X_val):
    m, n = X_train.shape

    self.P = 3 * np.random.rand(self.n_latent_features, m)
    self.Q = 3 * np.random.rand(self.n_latent_features, n)
    
    self.train_error = []
    self.val_error = []

    users, items = X_train.nonzero()
    
    for epoch in range(self.n_epochs):
        for u, i in zip(users, items):
            error = X_train[u, i] - self.predictions(self.P[:,u], self.Q[:,i])
            self.P[:, u] += self.learning_rate * (error * self.Q[:, i] - self.lmbda * self.P[:, u])
            self.Q[:, i] += self.learning_rate * (error * self.P[:, u] - self.lmbda * self.Q[:, i])

        train_rmse = rmse(self.predictions(self.P, self.Q), X_train)
        val_rmse = rmse(self.predictions(self.P, self.Q), X_val)
        self.train_error.append(train_rmse)
        self.val_error.append(val_rmse)
        
    return self
  
  def predict(self, X_train, user_index):
    y_hat = self.predictions(self.P, self.Q)
    predictions_index = np.where(X_train[user_index, :] == 0)[0]
    return y_hat[user_index, predictions_index].flatten()

In [ ]:
!pip install git+https://github.com/BirkhoffG/causalgraphicalmodels.git

## INITIAL ASSUMPTION

U → R: User preferences influence user-item interactions. Users are more likely to interact with items (e.g., listen to or rate them) if they align with their preferences.

I → R: Item characteristics influence user-item interactions. Items with certain characteristics (e.g., popular genres) are more likely to be interacted with by users.

U ← I: User preferences may also influence item characteristics. For example, if many users prefer a certain genre, artists producing that genre may become more popular.

In [ ]:
import causalgraphicalmodels as cgm
from causalgraphicalmodels import CausalGraphicalModel

# Define the causal relationships in the DAG
dag = CausalGraphicalModel(
    nodes=["U", "I", "R"],
    edges=[
        ("U", "R"),  # User preferences influence user-item interactions
        ("I", "R"),  # Item characteristics influence user-item interactions
        ("U", "I")   # User preferences influence item characteristics
    ]
)

# Plot the DAG
dag.draw()


In [ ]:
recommender = Recommender().fit(train, val)

In [ ]:
plt.plot(range(recommender.n_epochs), recommender.train_error, marker='o', label='Training Data');
plt.plot(range(recommender.n_epochs), recommender.val_error, marker='v', label='Validation Data');
plt.xlabel('Number of Epochs');
plt.ylabel('RMSE');
plt.legend()
plt.grid()
plt.show()

In [ ]:
user_id = 1236
user_index = ratings_df.index.get_loc(user_id)
predictions_index = np.where(train[user_index, :] == 0)[0]

rating_predictions = recommender.predict(train, user_index)

In [ ]:
def create_artist_ratings(artists_df, artists_index, ratings, n=10):
  artist_ids = ratings_df.columns[artists_index]
  artist_ratings = pd.DataFrame(data=dict(artistId=artist_ids, rating=ratings))
  top_n_artists = artist_ratings.sort_values("rating", ascending=False).head(n)
  
  artist_recommendations = artists_df[artists_df.id.isin(top_n_artists.artistId)].reset_index(drop=True)
  artist_recommendations['rating'] = pd.Series(top_n_artists.rating.values)
  return artist_recommendations.sort_values("rating", ascending=False)

In [ ]:
existing_ratings_index = np.where(train[user_index, :] > 0)[0]
existing_ratings = train[user_index, existing_ratings_index]

create_artist_ratings(artists, existing_ratings_index, existing_ratings)

In [ ]:
create_artist_ratings(artists, predictions_index, rating_predictions)

## KALMAN FILTERING

In [ ]:
# Sample 1% of the dataset
sampled_ap = ap.sample(frac=0.5, random_state=42)  # Adjust the fraction as needed

# Check the shape of the sampled dataset
print("Shape of sampled dataset:", sampled_ap.shape)

In [ ]:
sampled_ap.head()

In [ ]:
# Assign the normalized play counts to a new column
sampled_ap['playCountScaled'] = play_count_scaled

# Pivot the dataset
ratings_df = sampled_ap.pivot(index='userID', columns='artistID', values='playCountScaled').fillna(0)

# Convert ratings dataframe to numpy array
ratings = ratings_df.values

# Calculate sparsity
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("Sparsity of the sampled dataset: {:.2f}%".format(sparsity))

# Define constants for train-test split
MIN_USER_RATINGS = 35
DELETE_RATING_COUNT = 15

# Function to perform train-test split
def train_test_split(ratings):
    validation = np.zeros(ratings.shape)
    train = ratings.copy()

    for user in np.arange(ratings.shape[0]):
        if len(ratings[user,:].nonzero()[0]) >= MIN_USER_RATINGS:
            val_ratings = np.random.choice(
                ratings[user, :].nonzero()[0], 
                size=DELETE_RATING_COUNT,
                replace=False
            )
            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]
    return train, validation

# Perform train-test split
train, val = train_test_split(ratings)

# Display the dimensions of train and validation sets
print("Train set dimensions:", train.shape)
print("Validation set dimensions:", val.shape)

In [ ]:
class KalmanFilter:
    def __init__(self, alpha, beta, initial_state_mean, initial_state_covariance, transition_covariance, observation_covariance):
        self.alpha = alpha  # State transition coefficient
        self.beta = beta    # Observation coefficient
        
        # Initialize state mean and covariance
        self.state_mean = initial_state_mean
        self.state_covariance = initial_state_covariance
        
        # Covariance matrices
        self.transition_covariance = transition_covariance
        self.observation_covariance = observation_covariance
    
    def predict(self):
        # Predict the next state
        self.state_mean = self.alpha * self.state_mean
        self.state_covariance = self.alpha**2 * self.state_covariance + self.transition_covariance
        
    def update(self, observation):
        # Update the state estimate based on the new observation
        kalman_gain = self.state_covariance * self.beta / (self.beta**2 * self.state_covariance + self.observation_covariance)
        self.state_mean += kalman_gain * (observation - self.beta * self.state_mean)
        self.state_covariance -= kalman_gain * self.beta * self.state_covariance
        
    def filter(self, observations):
        filtered_states = []
        for observation in observations:
            self.predict()
            self.update(observation)
            filtered_states.append(self.state_mean)
        return np.array(filtered_states)

# Define model parameters
alpha = 0.8
beta = 1.0
initial_state_mean = 0
initial_state_covariance = 1
transition_covariance = 0.1
observation_covariance = 0.5


In [ ]:
# Initialize Kalman filter
kf = KalmanFilter(alpha, beta, initial_state_mean, initial_state_covariance, transition_covariance, observation_covariance)

# Prepare observations for each user
unique_user_ids = sampled_ap['userID'].unique()
filtered_states_by_user = {}

# Run Kalman filter for each user
for user_id in unique_user_ids:
    user_play_counts = sampled_ap[sampled_ap['userID'] == user_id]['playCountScaled'].values
    filtered_states = kf.filter(user_play_counts)
    filtered_states_by_user[user_id] = filtered_states

# Example of accessing filtered states for a specific user
example_user_id = unique_user_ids[0]
example_filtered_states = filtered_states_by_user[example_user_id]
print(f"Filtered States for User {example_user_id}:")
print(example_filtered_states)


In [ ]:
class LSSM:
    def __init__(self, n_epochs=200, n_latent_features=3, lmbda=0.1, learning_rate=0.001):
        self.n_epochs = n_epochs
        self.n_latent_features = n_latent_features
        self.lmbda = lmbda
        self.learning_rate = learning_rate

    def predictions(self, P, Q):
        return np.dot(P.T, Q)

    def fit(self, X_train, X_val):
        m, n = X_train.shape
        self.P = 3 * np.random.rand(self.n_latent_features, m)
        self.Q = 3 * np.random.rand(self.n_latent_features, n)
        self.train_error = []
        self.val_error = []

        for epoch in range(self.n_epochs):
            for u in range(m):
                for i in range(n):
                    if X_train[u, i] > 0:
                        error = X_train[u, i] - np.dot(self.P[:, u], self.Q[:, i])
                        self.P[:, u] += self.learning_rate * (error * self.Q[:, i] - self.lmbda * self.P[:, u])
                        self.Q[:, i] += self.learning_rate * (error * self.P[:, u] - self.lmbda * self.Q[:, i])

            # Calculate training and validation errors for the epoch
            train_predictions = self.predictions(self.P, self.Q)
            train_rmse = rmse(train_predictions, X_train)
            val_rmse = rmse(train_predictions, X_val)
            self.train_error.append(train_rmse)
            self.val_error.append(val_rmse)

            # Print epoch status
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Training RMSE = {train_rmse}, Validation RMSE = {val_rmse}")

        return self

    def mae(self, prediction, ground_truth):
        return np.mean(np.abs(prediction - ground_truth))

    def coverage(self, top_n, user_index):
        user_predictions = self.predictions(self.P, self.Q)[user_index, :]
        top_indices = np.argpartition(-user_predictions, top_n)[:top_n]
        coverage = len(np.where(user_predictions[top_indices] > 0)[0]) / top_n
        return coverage

In [ ]:
# Instantiate the LSSM class
lssm_model = LSSM(n_epochs=200, n_latent_features=3, lmbda=0.1, learning_rate=0.001)

# Fit the LSSM model to the training data
lssm_model.fit(train, val)

In [ ]:
# Plot the epoch error rates
plt.plot(range(lssm_model.n_epochs), lssm_model.train_error,marker='o', label='Training RMSE')
plt.plot(range(lssm_model.n_epochs), lssm_model.val_error, marker='v',label='Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.title('Epoch Error Rates')
plt.legend()
plt.show()

In [ ]:
# Extract learned latent features
user_latent_features = lssm_model.P
item_latent_features = lssm_model.Q

# Plot posterior distributions for user latent features
num_latent_features = user_latent_features.shape[0]
fig, axs = plt.subplots(num_latent_features, 1, figsize=(10, 5*num_latent_features))
fig.suptitle('Posterior Distributions of User Latent Features', fontsize=16)
for i in range(num_latent_features):
    axs[i].hist(user_latent_features[i], bins=30, color='skyblue', alpha=0.7)
    axs[i].set_title(f'Latent Feature {i+1}')
    axs[i].set_xlabel('Value')
    axs[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Plot posterior distributions for item latent features
num_latent_features = item_latent_features.shape[0]
fig, axs = plt.subplots(num_latent_features, 1, figsize=(10, 5*num_latent_features))
fig.suptitle('Posterior Distributions of Item Latent Features', fontsize=16)
for i in range(num_latent_features):
    axs[i].hist(item_latent_features[i], bins=30, color='lightgreen', alpha=0.7)
    axs[i].set_title(f'Latent Feature {i+1}')
    axs[i].set_xlabel('Value')
    axs[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# CF model RMSE values over 200 epochs
cf_training_rmse = [4.890360128948167, 1.2702533421384172, 0.7174904311685398, 0.5070622453084246, 0.3924680652571997, 
                    0.3181449014575357, 0.2652105794444253, 0.22534769181282227, 0.19421065104865096, 0.16925061945615885, 
                    0.14884739342135264, 0.13190996403077507, 0.11767160179823927, 0.10557551464961308, 0.09520672689783584, 
                    0.0862493444402953, 0.07845861406448355, 0.07164202973962444, 0.06564619505893841, 0.06034747102743732]

cf_validation_rmse = [4.975711230626356, 1.4333280645618143, 0.863349524878211, 0.6385653681501853, 0.5139215479802262, 
                      0.4319578110589442, 0.37274128584059374, 0.3274384280763896, 0.2914364856692566, 0.2620365486249346, 
                      0.23752690306874125, 0.21675653094477323, 0.19891689756955616, 0.18342062729072653, 0.16982946870117088, 
                      0.1578092305912475, 0.1471003810245595, 0.13749819777894653, 0.128838982896398, 0.12099025894534568]

# LSSM model RMSE values over 200 epochs
lssm_training_rmse = [6.366275485312182, 3.0989074724113026, 2.2561041575799727, 1.8650009341540403, 1.6357208856525307, 
                      1.4824025899326378, 1.3710483453437667, 1.2855193240543803, 1.2171352550399153, 1.1607869805397317, 
                      1.1132572177514872, 1.0724112948603983, 1.036772629119924, 1.0052845612309689, 0.9771692691237346, 
                      0.9518403445665427, 0.9288465327202755, 0.9078343397602808, 0.8885224809577605, 0.87068399346432]

lssm_validation_rmse = [6.366278091565758, 3.0989085548022093, 2.2561047834864705, 1.8650013146215385, 1.6357211254489372, 
                        1.4824027436889653, 1.3710484429282794, 1.2855193827173095, 1.2171352853065043, 1.160786989169136, 
                        1.1132572092882411, 1.072411272485641, 1.0367725951396047, 1.0052845173677931, 0.9771692166985678, 
                        0.9518402846151813, 0.928846466070993, 0.9078342670868176, 0.8885224028164473, 0.8706839103206843]

# Plotting RMSE values over epochs
epochs = range(0, len(cf_training_rmse) * 10, 10)  # Assuming an epoch is recorded every 10 steps

plt.figure(figsize=(12, 6))
plt.plot(epochs, cf_training_rmse, label='CF Training RMSE', color='blue')
plt.plot(epochs, cf_validation_rmse, label='CF Validation RMSE', color='red')
plt.plot(epochs, lssm_training_rmse, label='LSSM Training RMSE', color='green')
plt.plot(epochs, lssm_validation_rmse, label='LSSM Validation RMSE', color='orange')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.title('RMSE vs Epochs for CF and LSSM Recommender Models')
plt.legend()
plt.grid(True)
plt.show()
